In [393]:
%run "./funcoes_classes.ipynb"

In [2]:
import glob
import pandas as pd
import joblib


# Classificacao com regex

In [4]:
dataset =  get_datas()

the number of txt archive is: 31158
the number of txt archive is: 15078


In [5]:
dataset = dataframe(dataset)

In [6]:
dataset.head(3)

,texto
0,ESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE ARA...
1,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE L...
2,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO E RH\n...


In [7]:
df = regex(dataset)

In [8]:
df.head(3)

,texto,label_regex,status
0,ESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE ARA...,None,unclassified
1,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE L...,None,unclassified
2,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO E RH\n...,None,unclassified


In [9]:
len(df)

46236

In [12]:
df.to_pickle('./versiona_vida_funcional_sgd/dataset/df_regex_v0.pkl')

In [74]:
df = pd.read_pickle('./versiona_vida_funcional_sgd/dataset/dataset_treinado_v0.pkl')

In [13]:
lista = [
    'NOMEACAO DE CARGO EM COMISSAO',
    'EXONERACAO DE CARGO EFETIVO', 
    'EXONERACAO DE CARGO EM COMISSAO', 
    'NOMEACAO EM CARGO EFETIVO',
    'DEMISSAO',
    'APOSENTADORIA'
    ]

df = filtra_dataset(df, lista)

/tmp/ipykernel_16843/1009361865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outros['label_regex'] = 'OUTROS'


In [70]:
df.head(3)

,texto,label_regex,status
0,ESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE ARA...,None,unclassified
1,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE L...,None,unclassified
2,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO E RH\n...,None,unclassified


# Modelo e treino v1

In [15]:
# Vetorização TD-IDF e pipeline
sgd_pipeline = pipeline_vetorizacao_classificacao()

In [16]:
x = df.texto
y = df.label_regex

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.5, random_state=42, shuffle=True, stratify=y)

In [18]:
sgd_pipeline.fit(x_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='modified_huber'))])

In [19]:
sgd_pipeline.score(x_test, y_test)

0.9807785888077859

In [20]:
import joblib

joblib.dump(sgd_pipeline,'./versiona_vida_funcional_sgd/pipeline/sgd_pipeline_v1.pkl')

['./versiona_vida_funcional_sgd/pipeline/sgd_pipeline_v0.pkl']

In [21]:
y_pred_all = sgd_pipeline.predict(df.texto)

df['predict_classification'] = y_pred_all

y_pred_proba_all = sgd_pipeline.predict_proba(df.texto)

df['predict_proba_label'] = list(y_pred_proba_all)

In [22]:
df.to_pickle('./versiona_vida_funcional_sgd/dataset/dataset_treinado_v1.pkl')

In [23]:
df_erros = extraindo_erros_dataset(df, sgd_pipeline, 'label_regex', 'predict_classification', 'predict_proba_label')

A quantidade de dados classificados com erro neste treinamento é: 73 


/tmp/ipykernel_16843/2507650403.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erros['probabilities'] = list(erros_prob)


In [24]:
df_erros.head(3)

,texto,label_regex,status,predict_classification,predict_proba_label,probabilities
724,\nSERVICO AUTONOMO DE AGUA E ESGOTO - SAAE\nPO...,EXONERACAO DE CARGO EFETIVO,classified,OUTROS,"[0.013001499729490688, 0.0008605138297893195, ...","[0.013001499729490688, 0.0008605138297893195, ..."
2081,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nREVOG...,EXONERACAO DE CARGO EFETIVO,classified,OUTROS,"[0.0, 0.0, 0.43755851281146707, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.43755851281146707, 0.0, 0.0, 0.0,..."
3162,\nGABINETE DO PREFEITO\nPORTARIA No 145/2021\n...,EXONERACAO DE CARGO EFETIVO,classified,OUTROS,"[0.0, 0.0, 0.10494605047113713, 0.0, 0.3986850...","[0.0, 0.0, 0.10494605047113713, 0.0, 0.3986850..."


In [25]:
envio_rubrix(df_erros, sgd_pipeline, 'erros_rubrix_v1','DecisionTreeClassifier')

  0%|          | 0/73 [00:00<?, ?it/s]

73 records logged to http://localhost:6900/ws/rubrix/erros_rubrix_v1


In [43]:
df_corrigido = load_df_rubrix('erros_rubrix_v1')

2022-04-25 08:22:30.342 | WARNING  | rubrix.client.rubrix_client:load:310 - The argument 'as_pandas' in `rb.load` will be deprecated in the future, and we will always return a `Dataset`. To emulate the future behavior set `as_pandas=False`. To get a pandas DataFrame, call `Dataset.to_pandas()`


In [88]:
df_corrigido = None
df_erros2 = None

In [89]:
df_corrigido = pd.read_pickle('./versiona_vida_funcional_sgd/dataset/df_correcao_rubrix_v1.pkl')

In [63]:
sgd_pipeline = joblib.load('./versiona_vida_funcional_sgd/pipeline/sgd_pipeline_v1.pkl')

In [91]:
df_erros2 = pd.DataFrame()

In [92]:
for t, row in df_corrigido.iterrows():
    df_erros2 = df_erros2.append({'id': row['id'], 'label': row['annotation'], 'texto': row['inputs']['text']}, ignore_index=True)
#     print(row['id'], row['annotation'], row['inputs']['text'])

In [93]:
df_erros2

,id,label,texto
0,436.0,OUTROS,\nSECRETARIA MUNICIPAL DE RELACOES\nINSTITUCIO...
1,724.0,EXONERACAO DE CARGO EFETIVO,\nSERVICO AUTONOMO DE AGUA E ESGOTO - SAAE\nPO...
2,971.0,OUTROS,\nPREFEITURA MUNICIPAL DE BELO MONTE\nPORTARIA...
3,1924.0,OUTROS,\nSECRETARIA MUNICIPAL DE RELACOES\nINSTITUCIO...
4,2081.0,EXONERACAO DE CARGO EFETIVO,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nREVOG...
...,...,...,...
68,31125.0,APOSENTADORIA,\nINSTITUTO DE PREVIDENCIA DOS SERVIDORES\nPUB...
69,32560.0,EXONERACAO DE CARGO EM COMISSAO,\nGABINETE DO PREFEITO - GP\nPORTARIA. No. 205...
70,36613.0,OUTROS,\nCAMARA MUNICIPAL DE MACEIO - CMM\nCOMISSAO D...
71,38354.0,OUTROS,\nCONSELHO MUNICIPAL DOS DIREITOS DA PESSOA ID...


In [94]:
# Build the Rubrix records
records = [
    rb.TextClassificationRecord(
        id=idx,
        inputs={'id':row['id'], 'texto':row['texto']},
        annotation = row['label']

    )
    for idx, row in df_erros2.iterrows()
]

# Log the records
rb.log(records, name='erros_rubrix_v1', 
    tags={
        "dataset": "erros classificacao diarios oficiais 2021",
        "metodologia": "dados off diagonal da matriz de confusao e com probabilidade menor que 90%"
        },
    )


  0%|          | 0/73 [00:00<?, ?it/s]

73 records logged to http://localhost:6900/ws/rubrix/erros_rubrix_v1


BulkResponse(dataset='erros_rubrix_v1', processed=73, failed=0)

In [96]:
df_corrigido = load_df_rubrix('erros_rubrix_v1', 1)

2022-04-25 10:32:41.270 | WARNING  | rubrix.client.rubrix_client:load:310 - The argument 'as_pandas' in `rb.load` will be deprecated in the future, and we will always return a `Dataset`. To emulate the future behavior set `as_pandas=False`. To get a pandas DataFrame, call `Dataset.to_pandas()`


In [128]:
df_corrigido.head(3)

,inputs,prediction,prediction_agent,annotation,annotation_agent,multi_label,explanation,id,metadata,status,event_timestamp,metrics,search_keywords
0,"{'id': '436.0', 'texto': ' SECRETARIA MUNICIPA...",None,None,OUTROS,andre-pereira,False,None,0,{},Validated,None,{'text_length': 1039},None
1,"{'id': '724.0', 'texto': ' SERVICO AUTONOMO DE...",None,None,EXONERACAO DE CARGO EFETIVO,andre-pereira,False,None,1,{},Validated,None,{'text_length': 1004},None
2,"{'id': '971.0', 'texto': ' PREFEITURA MUNICIPA...",None,None,OUTROS,andre-pereira,False,None,2,{},Validated,None,{'text_length': 789},None


In [129]:
df_corrigido.annotation.isna().drop_duplicates()

0    False
Name: annotation, dtype: bool

In [140]:
for t, row in df_corrigido.iterrows():
    print(row['inputs']['id'], row['annotation'], row['inputs']['texto'])

436.0 OUTROS 
SECRETARIA MUNICIPAL DE RELACOES
INSTITUCIONAIS
PORTARIA No 494/2021
(De 15 de junho de 2021)
NOMEACAO DE AGENTES PUBLICOS EM
CARGOS EM COMISSAO NA ESFERA DA
ADMINISTRACAO DIRETA.
O PREFEITO DO MUNICIPIO DE MARAGOGI, Estado de
Alagoas, no uso das atribuicoes legais conferida pela Lei Organica do
Municipio, Lei ndeg 670/2019, de 08 de marco de 2019 e pela
Constituicao Federal.
RESOLVE
Art. 1deg Designar o Sr. JOSE GABRIEL MENDES DE
VASCONCELOS FERREIRA, inscrito no Cadastro de Pessoa
Fisica - CPF ndeg 064.881.434-33, para exercer as atribuicoes do Cargo
em Comissao de SECRETARIO MUNICIPAL DE MEIO
AMBIENTE E RECURSOS HIDRICOS, subordinado ao Chefe do
poder Executivo.
Art. 2deg Esta Portaria entrara em vigor na data de sua nomeacao,
revogadas as disposicoes em contrario.
De ciencia; publique-se; registre-se; e cumpra-se.
Gabinete do Prefeito do Municipio de Maragogi/AL, ao decimo
quinto dia do mes de junho de 2021.
FERNANDO SERGIO LIRA NETO
Prefeito
Publicado por:
Italo Jose

In [372]:
df_erros_v0 = None
df_erros_v0 = pd.DataFrame()

In [373]:
int(float('436.0'))

436

In [374]:
for t, row in df_corrigido.iterrows():
    df_erros_v0 = df_erros_v0.append({'id_item': int(float(row['inputs']['id'])), 'label': row['annotation'], 'texto': row['inputs']['texto']}, ignore_index=True)
#     print(row['id'], row['annotation'], row['inputs']['text'])

In [375]:
df_erros_v0.label.isna().drop_duplicates()

0    False
Name: label, dtype: bool

In [376]:
df_erros_v0.id_item

0       436.0
1       724.0
2       971.0
3      1924.0
4      2081.0
       ...   
68    31125.0
69    32560.0
70    36613.0
71    38354.0
72    42595.0
Name: id_item, Length: 73, dtype: float64

In [377]:
# for i in df.index:
#     if i in df_erros.index:
#         print(i)

In [451]:
df_corrigido.to_pickle('./versiona_vida_funcional_sgd/dataset/erros_rubrix_v1.pkl')

In [425]:
dataset_treinado_v1 = pd.read_pickle('./versiona_vida_funcional_sgd/dataset/dataset_treinado_v1.pkl')

In [426]:
dataset_treinado_v1['id_item'] = df.index

In [427]:
dataset_treinado_v1 = dataset_treinado_v1.rename(columns={'predict_classification': 'predict_classification_v1'})

In [428]:
dataset_treinado_v1

,texto,label_regex,status,predict_classification_v1,predict_proba_label,id_item
75,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",75
243,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE C...,NOMEACAO EM CARGO EFETIVO,classified,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.936737557555595, 0...",243
377,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",377
386,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",386
466,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",466
...,...,...,...,...,...,...
46220,\nGABINETE DE GOVERNANCA - GGOV\nPORTARIA No. ...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46220
46222,\nSECRETARIA MUNICIPAL DE INFRAESTRUTURA -\nSE...,OUTROS,classified,OUTROS,"[0.012449691342546122, 0.0, 0.0, 0.0, 0.0, 0.0...",46222
46228,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46228
46230,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46230


In [429]:
df_erros_v0 = df_erros_v0.rename(columns={'label': 'predict_classification_v1'})

In [430]:
df_erros_v0

,id_item,predict_classification_v1,texto
0,436.0,OUTROS,\nSECRETARIA MUNICIPAL DE RELACOES\nINSTITUCIO...
1,724.0,EXONERACAO DE CARGO EFETIVO,\nSERVICO AUTONOMO DE AGUA E ESGOTO - SAAE\nPO...
2,971.0,OUTROS,\nPREFEITURA MUNICIPAL DE BELO MONTE\nPORTARIA...
3,1924.0,OUTROS,\nSECRETARIA MUNICIPAL DE RELACOES\nINSTITUCIO...
4,2081.0,EXONERACAO DE CARGO EFETIVO,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nREVOG...
...,...,...,...
68,31125.0,APOSENTADORIA,\nINSTITUTO DE PREVIDENCIA DOS SERVIDORES\nPUB...
69,32560.0,EXONERACAO DE CARGO EM COMISSAO,\nGABINETE DO PREFEITO - GP\nPORTARIA. No. 205...
70,36613.0,OUTROS,\nCAMARA MUNICIPAL DE MACEIO - CMM\nCOMISSAO D...
71,38354.0,OUTROS,\nCONSELHO MUNICIPAL DOS DIREITOS DA PESSOA ID...


In [431]:
dataset_treinado_v1['merge_label_v1'] = dataset_treinado_v1['predict_classification_v1']

In [432]:
dataset_treinado_v1

,texto,label_regex,status,predict_classification_v1,predict_proba_label,id_item,merge_label_v1
75,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",75,APOSENTADORIA
243,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE C...,NOMEACAO EM CARGO EFETIVO,classified,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.936737557555595, 0...",243,NOMEACAO EM CARGO EFETIVO
377,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",377,APOSENTADORIA
386,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",386,APOSENTADORIA
466,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",466,APOSENTADORIA
...,...,...,...,...,...,...,...
46220,\nGABINETE DE GOVERNANCA - GGOV\nPORTARIA No. ...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46220,OUTROS
46222,\nSECRETARIA MUNICIPAL DE INFRAESTRUTURA -\nSE...,OUTROS,classified,OUTROS,"[0.012449691342546122, 0.0, 0.0, 0.0, 0.0, 0.0...",46222,OUTROS
46228,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46228,OUTROS
46230,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46230,OUTROS


In [433]:
df_erros_v0

,id_item,predict_classification_v1,texto
0,436.0,OUTROS,\nSECRETARIA MUNICIPAL DE RELACOES\nINSTITUCIO...
1,724.0,EXONERACAO DE CARGO EFETIVO,\nSERVICO AUTONOMO DE AGUA E ESGOTO - SAAE\nPO...
2,971.0,OUTROS,\nPREFEITURA MUNICIPAL DE BELO MONTE\nPORTARIA...
3,1924.0,OUTROS,\nSECRETARIA MUNICIPAL DE RELACOES\nINSTITUCIO...
4,2081.0,EXONERACAO DE CARGO EFETIVO,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nREVOG...
...,...,...,...
68,31125.0,APOSENTADORIA,\nINSTITUTO DE PREVIDENCIA DOS SERVIDORES\nPUB...
69,32560.0,EXONERACAO DE CARGO EM COMISSAO,\nGABINETE DO PREFEITO - GP\nPORTARIA. No. 205...
70,36613.0,OUTROS,\nCAMARA MUNICIPAL DE MACEIO - CMM\nCOMISSAO D...
71,38354.0,OUTROS,\nCONSELHO MUNICIPAL DOS DIREITOS DA PESSOA ID...


In [435]:
dataset_treinado_v1 = merge_dataset(dataset_treinado_v1, df_erros_v0, 1)

8219
8219
                                                   texto  \
724    \nSERVICO AUTONOMO DE AGUA E ESGOTO - SAAE\nPO...   
2081   \nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nREVOG...   
3162   \nGABINETE DO PREFEITO\nPORTARIA No 145/2021\n...   
3440   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE D...   
3640   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...   
4378   \nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nNA PO...   
5750   \nGABINETE DO PREFEITO\nLEI No 641/2021\n"ALTE...   
5868   \nGABINETE DO PREFEITO\nLEI No 1.171/2021, DE ...   
7816   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE P...   
8458   \nPREFEITURA MUNICIPAL DE INHAPI\nPORTARIA No ...   
10136  \nGABINETE DA PREFEITA\nPORTARIA\nPORTARIA Nde...   
13064  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
13069  \nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nPORTA...   
13095  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
13123  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
13502  \nPREFEITURA MUNICIPAL 

In [448]:
dataset_treinado_v1.to_pickle('./versiona_vida_funcional_sgd/dataset/dataset_merge_v1.pkl')

# Treino e correção v2

### Load

In [3]:
sgd_pipeline = joblib.load('./versiona_vida_funcional_sgd/pipeline/sgd_pipeline_v1.pkl')

In [4]:
dataset_merge_v1 = pd.read_pickle('./versiona_vida_funcional_sgd/dataset/dataset_merge_v1.pkl')

In [5]:
dataset_merge_v1

,texto,label_regex,status,predict_classification_v1,predict_proba_label,id_item,merge_label_v1
75,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",75,APOSENTADORIA
243,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE C...,NOMEACAO EM CARGO EFETIVO,classified,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.936737557555595, 0...",243,NOMEACAO EM CARGO EFETIVO
377,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",377,APOSENTADORIA
386,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",386,APOSENTADORIA
466,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",466,APOSENTADORIA
...,...,...,...,...,...,...,...
46220,\nGABINETE DE GOVERNANCA - GGOV\nPORTARIA No. ...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46220,OUTROS
46222,\nSECRETARIA MUNICIPAL DE INFRAESTRUTURA -\nSE...,OUTROS,classified,OUTROS,"[0.012449691342546122, 0.0, 0.0, 0.0, 0.0, 0.0...",46222,OUTROS
46228,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46228,OUTROS
46230,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46230,OUTROS


In [6]:
x = dataset_merge_v1.texto
y = dataset_merge_v1.merge_label_v1

In [7]:
dataset_treinado_v2, sgd_pipeline = train_dataset(sgd_pipeline, dataset_merge_v1, x, y, 2)

['NOMEACAO DE CARGO EM COMISSAO' 'OUTROS' 'OUTROS' ... 'OUTROS' 'OUTROS'
 'OUTROS']
[[0.         0.         0.         ... 1.         0.         0.        ]
 [0.28991136 0.         0.03455412 ... 0.         0.         0.67553452]
 [0.02855778 0.         0.         ... 0.         0.         0.97144222]
 ...
 [0.16933713 0.         0.         ... 0.         0.         0.83066287]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
0.9834549878345499


In [8]:
dataset_treinado_v2

,texto,label_regex,status,predict_classification_v1,predict_proba_label,id_item,merge_label_v1,predict_label_v2,predict_proba_label_v2
75,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",75,APOSENTADORIA,APOSENTADORIA,"[0.9465893910510942, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
243,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE C...,NOMEACAO EM CARGO EFETIVO,classified,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.936737557555595, 0...",243,NOMEACAO EM CARGO EFETIVO,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.9125084610711652, ..."
377,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",377,APOSENTADORIA,APOSENTADORIA,"[0.9754928750098415, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
386,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",386,APOSENTADORIA,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
466,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",466,APOSENTADORIA,APOSENTADORIA,"[0.975372651938113, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
...,...,...,...,...,...,...,...,...,...
46220,\nGABINETE DE GOVERNANCA - GGOV\nPORTARIA No. ...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46220,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
46222,\nSECRETARIA MUNICIPAL DE INFRAESTRUTURA -\nSE...,OUTROS,classified,OUTROS,"[0.012449691342546122, 0.0, 0.0, 0.0, 0.0, 0.0...",46222,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
46228,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46228,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
46230,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46230,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"


In [10]:
df_erros = extraindo_erros_dataset(dataset_treinado_v2, sgd_pipeline, 'merge_label_v1', 'predict_label_v2', 'predict_proba_label_v2')

A quantidade de dados classificados com erro neste treinamento é: 60 


/tmp/ipykernel_2589/2507650403.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erros['probabilities'] = list(erros_prob)


In [62]:
envio_rubrix(df_erros, sgd_pipeline, 2,'SGDClassifier')

  0%|          | 0/60 [00:00<?, ?it/s]

60 records logged to http://localhost:6900/ws/rubrix/erros_rubrix_v2


In [11]:
df_erros

,texto,label_regex,status,predict_classification_v1,predict_proba_label,id_item,merge_label_v1,predict_label_v2,predict_proba_label_v2,probabilities
2081,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nREVOG...,EXONERACAO DE CARGO EFETIVO,classified,OUTROS,"[0.0, 0.0, 0.43755851281146707, 0.0, 0.0, 0.0,...",2081,EXONERACAO DE CARGO EFETIVO,OUTROS,"[0.0, 0.0, 0.4641399100701377, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.4641399100701377, 0.0, 0.0, 0.0, ..."
3440,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE D...,DEMISSAO,classified,EXONERACAO DE CARGO EFETIVO,"[0.018250122916056762, 0.3462603944513642, 0.4...",3440,DEMISSAO,EXONERACAO DE CARGO EFETIVO,"[0.019185989839676384, 0.28237731305028607, 0....","[0.019185989839676384, 0.28237731305028607, 0...."
3524,\nGABINETE DO PREFEITO\nPORTARIA No. 720 DE 01...,EXONERACAO DE CARGO EFETIVO,classified,EXONERACAO DE CARGO EFETIVO,"[0.0, 0.0, 0.875046590169614, 0.02969302263386...",3524,EXONERACAO DE CARGO EFETIVO,OUTROS,"[0.0, 0.0, 0.46092768623657965, 0.056785980792...","[0.0, 0.0, 0.46092768623657965, 0.056785980792..."
3640,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,OUTROS,"[0.1147854904612249, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3640,APOSENTADORIA,OUTROS,"[0.22347935630974197, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.22347935630974197, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4378,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nNA PO...,APOSENTADORIA,classified,OUTROS,"[0.39004368155654295, 0.0, 0.0, 0.0, 0.0, 0.0,...",4378,APOSENTADORIA,OUTROS,"[0.23583992839934004, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.23583992839934004, 0.0, 0.0, 0.0, 0.0, 0.0,..."
8510,\nGABINETE DO PREFEITO\nDECRETO No 003/2021\n(...,EXONERACAO DE CARGO EM COMISSAO,classified,EXONERACAO DE CARGO EM COMISSAO,"[0.0, 0.0, 0.0, 0.9091097875608011, 0.09089021...",8510,EXONERACAO DE CARGO EM COMISSAO,OUTROS,"[0.0, 0.0, 0.22127864947562592, 0.183910003364...","[0.0, 0.0, 0.22127864947562592, 0.183910003364..."
8890,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,NOMEACAO DE CARGO EM COMISSAO,classified,NOMEACAO DE CARGO EM COMISSAO,"[0.0, 0.0, 0.0, 0.16983111538593157, 0.7201632...",8890,NOMEACAO DE CARGO EM COMISSAO,OUTROS,"[0.0, 0.0, 0.0, 0.23421897091017385, 0.0198472...","[0.0, 0.0, 0.0, 0.23421897091017385, 0.0198472..."
8926,\nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nPORTA...,NOMEACAO DE CARGO EM COMISSAO,classified,NOMEACAO DE CARGO EM COMISSAO,"[0.0, 0.0, 0.0, 0.03954166263744444, 0.8955420...",8926,NOMEACAO DE CARGO EM COMISSAO,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.2861791829845296, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.2861791829845296, 0.0, ..."
10109,\nGABINETE DO PREFEITO\nPORTARIA PMT No 509/20...,APOSENTADORIA,classified,APOSENTADORIA,"[0.6013160186723658, 0.0, 0.1901060715145869, ...",10109,APOSENTADORIA,OUTROS,"[0.2663377056644408, 0.0, 0.2889420473178146, ...","[0.2663377056644408, 0.0, 0.2889420473178146, ..."
10824,\nGABINETE DA PREFEITA\nPREFEITURA MUNICIPAL D...,APOSENTADORIA,classified,APOSENTADORIA,"[0.5857446984910611, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10824,APOSENTADORIA,OUTROS,"[0.47948849983147984, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.47948849983147984, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [103]:
df_corrigido = load_df_rubrix('erros_rubrix_v2', 2)

2022-04-25 15:07:52.035 | WARNING  | rubrix.client.rubrix_client:load:310 - The argument 'as_pandas' in `rb.load` will be deprecated in the future, and we will always return a `Dataset`. To emulate the future behavior set `as_pandas=False`. To get a pandas DataFrame, call `Dataset.to_pandas()`


In [197]:
df_corrigido.head(1)

,id_item,label,texto
0,1064,APOSENTADORIA,\nPROCURADORIA GERAL DO MUNICIPIO\nDECRETO No ...


In [294]:
dataset_merge_v2 = merge_dataset(dataset_treinado_v2, df_corrigido, 2)

8219
8219
                                                   texto  \
2081   \nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nREVOG...   
3440   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE D...   
3524   \nGABINETE DO PREFEITO\nPORTARIA No. 720 DE 01...   
3640   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...   
4378   \nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nNA PO...   
8510   \nGABINETE DO PREFEITO\nDECRETO No 003/2021\n(...   
8890   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...   
8926   \nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nPORTA...   
10109  \nGABINETE DO PREFEITO\nPORTARIA PMT No 509/20...   
10824  \nGABINETE DA PREFEITA\nPREFEITURA MUNICIPAL D...   
13095  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
13123  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
13135  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
13454  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
13523  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
14521  \nSECRETARIA MUNICIPAL 

In [295]:
dataset_merge_v2

,texto,label_regex,status,predict_classification_v1,predict_proba_label,id_item,merge_label_v1,predict_label_v2,predict_proba_label_v2,merge_label_v2
75,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",75,APOSENTADORIA,APOSENTADORIA,"[0.9465893910510942, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA
243,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE C...,NOMEACAO EM CARGO EFETIVO,classified,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.936737557555595, 0...",243,NOMEACAO EM CARGO EFETIVO,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.9125084610711652, ...",NOMEACAO EM CARGO EFETIVO
377,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",377,APOSENTADORIA,APOSENTADORIA,"[0.9754928750098415, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA
386,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",386,APOSENTADORIA,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",APOSENTADORIA
466,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",466,APOSENTADORIA,APOSENTADORIA,"[0.975372651938113, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",APOSENTADORIA
...,...,...,...,...,...,...,...,...,...,...
46220,\nGABINETE DE GOVERNANCA - GGOV\nPORTARIA No. ...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46220,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS
46222,\nSECRETARIA MUNICIPAL DE INFRAESTRUTURA -\nSE...,OUTROS,classified,OUTROS,"[0.012449691342546122, 0.0, 0.0, 0.0, 0.0, 0.0...",46222,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS
46228,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46228,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS
46230,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46230,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS


# Treino e correção v3

In [296]:
sgd_pipeline = joblib.load('./versiona_vida_funcional_sgd/pipeline/sgd_pipeline_v2.pkl')

In [297]:
dataset_merge_v2 = pd.read_pickle('./versiona_vida_funcional_sgd/dataset/dataset_merge_v2.pkl')

In [299]:
dataset_merge_v2

,texto,label_regex,status,predict_classification_v1,predict_proba_label,id_item,merge_label_v1,predict_label_v2,predict_proba_label_v2,merge_label_v2
75,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",75,APOSENTADORIA,APOSENTADORIA,"[0.9465893910510942, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA
243,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE C...,NOMEACAO EM CARGO EFETIVO,classified,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.936737557555595, 0...",243,NOMEACAO EM CARGO EFETIVO,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.9125084610711652, ...",NOMEACAO EM CARGO EFETIVO
377,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",377,APOSENTADORIA,APOSENTADORIA,"[0.9754928750098415, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA
386,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",386,APOSENTADORIA,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",APOSENTADORIA
466,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",466,APOSENTADORIA,APOSENTADORIA,"[0.975372651938113, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",APOSENTADORIA
...,...,...,...,...,...,...,...,...,...,...
46220,\nGABINETE DE GOVERNANCA - GGOV\nPORTARIA No. ...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46220,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS
46222,\nSECRETARIA MUNICIPAL DE INFRAESTRUTURA -\nSE...,OUTROS,classified,OUTROS,"[0.012449691342546122, 0.0, 0.0, 0.0, 0.0, 0.0...",46222,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS
46228,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46228,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS
46230,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46230,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS


In [300]:
x = dataset_merge_v2.texto
y = dataset_merge_v2.merge_label_v2

In [301]:
dataset_treinado_v3, sgd_pipeline = train_dataset(sgd_pipeline, dataset_merge_v2, x, y, 3)

['NOMEACAO DE CARGO EM COMISSAO' 'OUTROS' 'OUTROS' ... 'OUTROS' 'OUTROS'
 'OUTROS']
[[0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.02471475 0.         0.97528525]
 [0.         0.         0.         ... 0.         0.         1.        ]
 ...
 [0.         0.         0.         ... 0.         0.00880874 0.99119126]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.00146512 ... 0.         0.         0.99853488]]
0.9868613138686131


In [302]:
dataset_treinado_v3

,texto,label_regex,status,predict_classification_v1,predict_proba_label,id_item,merge_label_v1,predict_label_v2,predict_proba_label_v2,merge_label_v2,predict_label_v3,predict_proba_label_v3
75,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",75,APOSENTADORIA,APOSENTADORIA,"[0.9465893910510942, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA,APOSENTADORIA,"[0.8023887125640096, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
243,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE C...,NOMEACAO EM CARGO EFETIVO,classified,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.936737557555595, 0...",243,NOMEACAO EM CARGO EFETIVO,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.9125084610711652, ...",NOMEACAO EM CARGO EFETIVO,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.849153478569961, 0..."
377,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",377,APOSENTADORIA,APOSENTADORIA,"[0.9754928750098415, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA,APOSENTADORIA,"[0.9973278106730926, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
386,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",386,APOSENTADORIA,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",APOSENTADORIA,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
466,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",466,APOSENTADORIA,APOSENTADORIA,"[0.975372651938113, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",APOSENTADORIA,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
46220,\nGABINETE DE GOVERNANCA - GGOV\nPORTARIA No. ...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46220,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
46222,\nSECRETARIA MUNICIPAL DE INFRAESTRUTURA -\nSE...,OUTROS,classified,OUTROS,"[0.012449691342546122, 0.0, 0.0, 0.0, 0.0, 0.0...",46222,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
46228,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46228,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
46230,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46230,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"


In [303]:
df_erros = extraindo_erros_dataset(dataset_treinado_v3, sgd_pipeline, 'merge_label_v2', 'predict_label_v3', 'predict_proba_label_v3')

A quantidade de dados classificados com erro neste treinamento é: 49 


/tmp/ipykernel_2589/2507650403.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erros['probabilities'] = list(erros_prob)


In [306]:
envio_rubrix(df_erros, sgd_pipeline, 3,'SGDClassifier')

  0%|          | 0/49 [00:00<?, ?it/s]

49 records logged to http://localhost:6900/ws/rubrix/erros_rubrix_v3


In [307]:
df_corrigido = load_df_rubrix('erros_rubrix_v3', 3)

2022-04-25 16:31:01.487 | WARNING  | rubrix.client.rubrix_client:load:310 - The argument 'as_pandas' in `rb.load` will be deprecated in the future, and we will always return a `Dataset`. To emulate the future behavior set `as_pandas=False`. To get a pandas DataFrame, call `Dataset.to_pandas()`


In [309]:
df_corrigido.head(1)

,id_item,label,texto
0,971,OUTROS,\nPREFEITURA MUNICIPAL DE BELO MONTE\nPORTARIA...


In [310]:
dataset_merge_v3 = merge_dataset(dataset_treinado_v3, df_corrigido, 3)

8219
8219
                                                   texto  \
2081   \nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nREVOG...   
3440   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE D...   
3524   \nGABINETE DO PREFEITO\nPORTARIA No. 720 DE 01...   
3640   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...   
4072   \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
4210   \nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nRATIF...   
4378   \nSECRETARIA MUNICIPAL DE ADMINISTRACAO\nNA PO...   
8110   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...   
8400   \nSECRETARIA MUNICIPAL DE ADMINISTRACAO E\nREC...   
8890   \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...   
10109  \nGABINETE DO PREFEITO\nPORTARIA PMT No 509/20...   
10802  \nGABINETE DO PREFEITO\nPORTARIA GP No 276/202...   
12118  \nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE S...   
13095  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
13123  \nPREFEITURA MUNICIPAL DE ARAPIRACA\nPORTARIA ...   
13135  \nPREFEITURA MUNICIPAL 

In [374]:
dataset_merge_v3

,texto,label_regex,status,predict_classification_v1,predict_proba_label,id_item,merge_label_v1,predict_label_v2,predict_proba_label_v2,merge_label_v2,predict_label_v3,predict_proba_label_v3,merge_label_v3
75,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",75,APOSENTADORIA,APOSENTADORIA,"[0.9465893910510942, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA,APOSENTADORIA,"[0.8023887125640096, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA
243,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE C...,NOMEACAO EM CARGO EFETIVO,classified,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.936737557555595, 0...",243,NOMEACAO EM CARGO EFETIVO,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.9125084610711652, ...",NOMEACAO EM CARGO EFETIVO,NOMEACAO EM CARGO EFETIVO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.849153478569961, 0...",NOMEACAO EM CARGO EFETIVO
377,\nESTADO DE ALAGOAS\nPREFEITURA MUNICIPAL DE M...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",377,APOSENTADORIA,APOSENTADORIA,"[0.9754928750098415, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA,APOSENTADORIA,"[0.9973278106730926, 0.0, 0.0, 0.0, 0.0, 0.0, ...",APOSENTADORIA
386,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",386,APOSENTADORIA,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",APOSENTADORIA,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",APOSENTADORIA
466,\nPREFEITURA MUNICIPAL DE MAR VERMELHO\nPORTAR...,APOSENTADORIA,classified,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",466,APOSENTADORIA,APOSENTADORIA,"[0.975372651938113, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",APOSENTADORIA,APOSENTADORIA,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",APOSENTADORIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46220,\nGABINETE DE GOVERNANCA - GGOV\nPORTARIA No. ...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46220,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS
46222,\nSECRETARIA MUNICIPAL DE INFRAESTRUTURA -\nSE...,OUTROS,classified,OUTROS,"[0.012449691342546122, 0.0, 0.0, 0.0, 0.0, 0.0...",46222,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS
46228,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46228,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS
46230,\nSECRETARIA MUNICIPAL DE GESTAO - SEMGE\nPORT...,OUTROS,classified,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",46230,OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS,OUTROS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",OUTROS


In [317]:
# sgd_pipeline = joblib.load('./versiona_vida_funcional_sgd/pipeline/sgd_pipeline_v2.pkl')

# dataset_merge_v2 = pd.read_pickle('./versiona_vida_funcional_sgd/dataset/dataset_merge_v2.pkl')


# x = dataset_merge_v2.texto
# y = dataset_merge_v2.merge_label_v2

# dataset_treinado_v3, sgd_pipeline = train_dataset(sgd_pipeline, dataset_merge_v2, x, y, 3)

# df_erros = extraindo_erros_dataset(dataset_treinado_v3, sgd_pipeline, 'merge_label_v2', 'predict_label_v3', 'predict_proba_label_v3')


# envio_rubrix(df_erros, sgd_pipeline, 3,'SGDClassifier')


# df_corrigido = load_df_rubrix('erros_rubrix_v3', 3)


# dataset_merge_v3 = merge_dataset(dataset_treinado_v3, df_corrigido, 3)


# Treino e correção v4

In [395]:
dataset_treinado_v4 = treina_e_envia_erros(4, 3)

['NOMEACAO DE CARGO EM COMISSAO' 'OUTROS' 'OUTROS' ... 'OUTROS' 'OUTROS'
 'OUTROS']
[[0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.0191784  0.         0.9808216 ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 ...
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.00496309 0.         0.         ... 0.         0.         0.99503691]
 [0.         0.         0.         ... 0.         0.         1.        ]]
0.9902676399026764
A quantidade de dados classificados com erro neste treinamento é: 44 


  0%|          | 0/44 [00:00<?, ?it/s]

GenericApiError: Rubrix server returned an error with http status: 500
Error details: [{'code': 'opensearchpy.exceptions.ConnectionError', 'params': {}}]

In [ ]:
dataset_merge_v4 = recebe_dados_rubrix_e_faz_merge(dataset_treinado_v4, 4)